In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/val-data/validation_formatted 1.csv


In [ ]:
%pip install -U sentence-transformers transformers datasets

In [3]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Load the model
model = SentenceTransformer('all-mpnet-base-v2')

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import pandas as pd
df = pd.read_csv("/kaggle/input/val-data/validation_formatted 1.csv")
df.head()

,invoice_id,invoice_desc,PO_line_desc,true_mapping
0,INV-49266647,"['25103 ICE CREAM MANGO SORBETTO TRAY', 'ICE C...","['ICE CREAM MANGO SORBETTO TRAY', 'ICE CREAM G...",[{'25103 ICE CREAM MANGO SORBETTO TRAY': 'ICE ...
1,INV-49266660,"['ICE CREAM GELATO MACKINAC FDGE 3/5LT', 'ICE ...","['ICE CREAM GELATO MACKINAC FDGE', 'ICE CREAM ...",[{'ICE CREAM GELATO MACKINAC FDGE 3/5LT': 'ICE...
2,INV-49266724,['Scotch-Brite Quick Clean Griddle Liquid 700-...,"['CLEANER GRL QUICKCLEAN PCKT', 'PAD SCOUR COM...",[{'Scotch-Brite Quick Clean Griddle Liquid 700...
3,INV-49267382,"['Jersey Mikes Mini Wheat Sub', 'Jersey Mikes'...","['ROLL MINI WHEAT SUB', 'ROLL MINI WHT SUB', '...",[{'Jersey Mikes Mini Wheat Sub': 'ROLL MINI WH...
4,INV-49267510,"['CASES BEEF GRND CHUCK PATTY 4oz', '14 CASES ...","['BEEF GRND CHUCK PATTY 4OZ', 'BEEF GRND THICK...",[{'CASES BEEF GRND CHUCK PATTY 4oz': 'BEEF GRN...


In [5]:
import re

def clean_text_v6(text):
    # Remove non-alphanumeric characters except spaces and commas
    text = re.sub(r'[^a-zA-Z0-9\s,]', '', text)
    # Normalize spaces
    text = re.sub(r'\s+', ' ', text)
    # Strip leading/trailing spaces
    text = text.strip()
    # Merge single letters separated by commas or spaces
    text = re.sub(r'\b([a-zA-Z])(,\s*|\s+)([a-zA-Z])(?:,\s*|\s+)([a-zA-Z])(?:,\s*|\s+)([a-zA-Z])\b', 
                   lambda x: ''.join(x.groups()), text)
    # Repeat for two or three letter sequences
    text = re.sub(r'\b([a-zA-Z])(?:,\s*|\s+)([a-zA-Z])(?:,\s*|\s+)?([a-zA-Z])?\b',
                  lambda x: ''.join(filter(None, x.groups())), text)
    return text.lower().strip()

# Apply the cleaning function to your DataFrame
df['invoice_desc_clean'] = df['invoice_desc'].apply(clean_text_v6)
df['PO_line_desc_clean'] = df['PO_line_desc'].apply(clean_text_v6)

In [6]:
# Verify the cleaned values
df[['invoice_desc_clean', 'PO_line_desc_clean']].head()

,invoice_desc_clean,PO_line_desc_clean
0,"25103 ice cream mango sorbetto tray, ice cream...","ice cream mango sorbetto tray, ice cream gelat..."
1,"ice cream gelato mackinac fdge 35lt, ice cream...","ice cream gelato mackinac fdge, ice cream crem..."
2,scotchbrite quick clean griddle liquid 70040 3...,"cleaner grl quickclean pckt, pad scour comm ge..."
3,"jersey mikes mini wheat sub, jersey mikes, jer...","roll mini wheat sub, roll mini wht sub, roll w..."
4,"cases beef grnd chuck patty 4oz, 14 cases beef...","beef grnd chuck patty 4oz, beef grnd thick pat..."


In [7]:
# Apply the cleaning function to invoice_desc and PO_line_desc
import ast
def clean_column_v6(desc):
    # Splitting by commas but also handling cases with inconsistent spaces
    desc = ast.literal_eval(desc)
    # Clean each item in the list
    cleaned_desc = [clean_text_v6(item.strip()) for item in desc]
    return ', '.join(cleaned_desc)

# Clean invoice_desc and PO_line_desc columns
df['invoice_desc_clean'] = df['invoice_desc'].apply(clean_column_v6)
df['PO_line_desc_clean'] = df['PO_line_desc'].apply(clean_column_v6)

In [8]:
# Preprocess the true_mapping column with enhanced cleaning
def clean_mapping_v6(mapping_str):
    clean_mapping_dict = []
    mappings = eval(mapping_str)  # Convert string to dict
    for mapping in mappings:
        invoice_item = list(mapping.keys())[0]
        purchase_item = mapping[invoice_item]
        # Clean both invoice and purchase items
        clean_invoice_item = clean_text_v6(invoice_item)
        clean_purchase_item = clean_text_v6(purchase_item)
        clean_mapping_dict.append({clean_invoice_item: clean_purchase_item})
    return clean_mapping_dict

df['true_mapping_clean'] = df['true_mapping'].apply(clean_mapping_v6)

In [9]:
def check_mapping_advanced(row):
    for mapping in row['true_mapping_clean']:
        invoice_item = list(mapping.keys())[0]
        purchase_item = mapping[invoice_item]

        # Ensure both cleaned invoice and purchase items exist in their respective cleaned columns
        if invoice_item not in row['invoice_desc_clean'] or purchase_item not in row['PO_line_desc_clean']:
            return False
    return True

# Apply the mapping validation check
df_cleaned = df[['invoice_desc_clean', 'PO_line_desc_clean', 'true_mapping_clean']].copy()
df_cleaned['mapping_valid'] = df_cleaned.apply(check_mapping_advanced, axis=1)

# Count valid and invalid mappings
print(df_cleaned['mapping_valid'].value_counts())

mapping_valid
True    946
Name: count, dtype: int64


In [10]:
df_cleaned.head()

,invoice_desc_clean,PO_line_desc_clean,true_mapping_clean,mapping_valid
0,"25103 ice cream mango sorbetto tray, ice cream...","ice cream mango sorbetto tray, ice cream gelat...",[{'25103 ice cream mango sorbetto tray': 'ice ...,True
1,"ice cream gelato mackinac fdge 35lt, ice cream...","ice cream gelato mackinac fdge, ice cream crem...",[{'ice cream gelato mackinac fdge 35lt': 'ice ...,True
2,scotchbrite quick clean griddle liquid 70040 3...,"cleaner grl quickclean pckt, pad scour comm ge...",[{'scotchbrite quick clean griddle liquid 7004...,True
3,"jersey mikes mini wheat sub, jersey mikes, jer...","roll mini wheat sub, roll mini wht sub, roll w...",[{'jersey mikes mini wheat sub': 'roll mini wh...,True
4,"cases beef grnd chuck patty 4oz, 14 cases beef...","beef grnd chuck patty 4oz, beef grnd thick pat...",[{'cases beef grnd chuck patty 4oz': 'beef grn...,True


In [11]:
def clean_text(text):
    # Normalize spaces and lowercase
    return ' '.join(text.split()).lower()

df_cleaned['invoice_desc_clean'] = df_cleaned['invoice_desc_clean'].apply(clean_text)
df_cleaned['PO_line_desc_clean'] = df_cleaned['PO_line_desc_clean'].apply(clean_text)

In [12]:
print(df_cleaned["invoice_desc_clean"][11])
print(df_cleaned["PO_line_desc_clean"][11])
print(df_cleaned["true_mapping_clean"][11])

bass striped fil farm skon frs usa, halibut filet skinon fresh usa, salmon fil scottish scaled scotland
bass striped fil farm skon frs, halibut filet frsh, salmon fil scottish scaled
[{'bass striped fil farm skon frs usa': 'bass striped fil farm skon frs'}, {'halibut filet skinon fresh usa': 'halibut filet frsh'}, {'salmon fil scottish scaled scotland': 'salmon fil scottish scaled'}]


In [13]:
# # First install spacy and download a pre-trained model
# !pip install spacy
# !python -m spacy download en_core_web_md

In [14]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Function to remove stop words and replace synonyms
def preprocess_text(text, stop_words):
    # Tokenize the text (split on spaces)
    tokens = text.lower().split()
    # Remove stop words
    tokens = [word for word in tokens if word not in stop_words]
    # Rejoin the tokens into a string
    return ' '.join(tokens)

# Apply the preprocessing to all three columns
df_cleaned['invoice_desc_clean'] = df_cleaned['invoice_desc_clean'].apply(
    lambda x: preprocess_text(x, stop_words)
)

df_cleaned['PO_line_desc_clean'] = df_cleaned['PO_line_desc_clean'].apply(
    lambda x: preprocess_text(x, stop_words)
)

# For true_mapping_clean, ensure it processes both the keys and values in the dictionary
def preprocess_mapping(mapping, stop_words):
    # Iterate through the list of dictionaries and preprocess both keys and values
    processed_mapping = []
    for pair in mapping:
        processed_pair = {
            preprocess_text(k, stop_words): preprocess_text(v, stop_words) 
            for k, v in pair.items()
        }
        processed_mapping.append(processed_pair)
    return processed_mapping

df_cleaned['true_mapping_clean'] = df_cleaned['true_mapping_clean'].apply(
    lambda mapping: preprocess_mapping(mapping, stop_words)
)

In [16]:
df_cleaned

,invoice_desc_clean,PO_line_desc_clean,true_mapping_clean,mapping_valid
0,"25103 ice cream mango sorbetto tray, ice cream...","ice cream mango sorbetto tray, ice cream gelat...",[{'25103 ice cream mango sorbetto tray': 'ice ...,True
1,"ice cream gelato mackinac fdge 35lt, ice cream...","ice cream gelato mackinac fdge, ice cream crem...",[{'ice cream gelato mackinac fdge 35lt': 'ice ...,True
2,scotchbrite quick clean griddle liquid 70040 3...,"cleaner grl quickclean pckt, pad scour comm ge...",[{'scotchbrite quick clean griddle liquid 7004...,True
3,"jersey mikes mini wheat sub, jersey mikes, jer...","roll mini wheat sub, roll mini wht sub, roll w...",[{'jersey mikes mini wheat sub': 'roll mini wh...,True
4,"cases beef grnd chuck patty 4oz, 14 cases beef...","beef grnd chuck patty 4oz, beef grnd thick pat...",[{'cases beef grnd chuck patty 4oz': 'beef grn...,True
...,...,...,...,...
941,"snow crab 58 oz 30, snow crab 58 oz 30, 2630 p...","crab snow clstr 58 oz jcs, crab snow clstr 58 ...",[{'snow crab 58 oz 30': 'crab snow clstr 58 oz...,True
942,"snow crab 58 oz 30, charthouse, scallops u10 3...","crab snow clstr 58 oz jcs, lobster tail 67z sr...",[{'snow crab 58 oz 30': 'crab snow clstr 58 oz...,True
943,"pork ground raw, pork shank osso bucco, pork b...","pork ground raw, pork shank osso bucco 3in gap...","[{'pork ground raw': 'pork ground raw'}, {'por...",True
944,"bag food liner bu, bag food liner","bag food liner bun sheet, bag food liner hotel...",[{'bag food liner bu': 'bag food liner bun she...,True


In [18]:
# Generate embeddings for invoice and purchase order descriptions
invoice_embeddings = model.encode(df['invoice_desc_clean'].tolist(), convert_to_tensor=True)
po_embeddings = model.encode(df['PO_line_desc_clean'].tolist(), convert_to_tensor=True)

# View the embeddings
print("Invoice Embeddings Shape:", invoice_embeddings.shape)
print("Purchase Order Embeddings Shape:", po_embeddings.shape)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Invoice Embeddings Shape: torch.Size([946, 768])
Purchase Order Embeddings Shape: torch.Size([946, 768])


In [19]:
print(df_cleaned["invoice_desc_clean"][547])
print(df_cleaned["PO_line_desc_clean"][547])
print(df_cleaned["true_mapping_clean"][547])

9 oz squat tmbl sysco 2025 4023073, 10 oz tall tmbl sysco 2025 4023081, 5 oz squat tumbler 2050, ecosense psm cutlery kit white, part descr lid clear 5010, 6 black cw plate 1018 sysco, 6 clear cw plate 1018 sysco
cup plas clr squat 9 oz rigid, cup plas clr tall 10oz rgd, cup plas clr 5 oz, kit cutlery f,skn plantstrch, lid plas clr 832 oz, plate plas blk 6 rgd clswr, plate plas clr 6 rgd clswr
[{'9 oz squat tmbl sysco 2025 4023073': 'cup plas clr squat 9 oz rigid'}, {'10 oz tall tmbl sysco 2025 4023081': 'cup plas clr tall 10oz rgd'}, {'5 oz squat tumbler 2050': 'cup plas clr 5 oz'}, {'ecosense psm cutlery kit white': 'kit cutlery f,skn plantstrch'}, {'part descr lid clear 5010': 'lid plas clr 832 oz'}, {'6 black cw plate 1018 sysco': 'plate plas blk 6 rgd clswr'}, {'6 clear cw plate 1018 sysco': 'plate plas clr 6 rgd clswr'}]


In [28]:
inv_lst = []
pov_lst = []
for inv in df["invoice_desc_clean"]:
    inv_lst.append(inv.split(", "))
    
for po in df["PO_line_desc_clean"]:
    pov_lst.append(po.split(", "))

In [29]:
inv_lst[:2]

[['25103 ice cream mango sorbetto tray',
  'ice cream gelato mackinac fdge 35lt',
  'ice cream cookiecrm dutch fud 35lt',
  'ice cream gelato butr pecan 3 5lt',
  'ice cream sorbet lemoncello 35 lt',
  'ice cream gelato choc nd vegan 35lt'],
 ['ice cream gelato mackinac fdge 35lt',
  'ice cream creme de minthe chip 35lt',
  'ice cream gelato van bean 3 5lt',
  'ice cream gelato rd velvet 35lt',
  'ice cream gelato peanut btr 35lt',
  'ice cream gelato swiss choc 3 5lt',
  'ice cream gelato straw frsh 35lt']]

In [31]:
pov_lst[:2]

[['ice cream mango sorbetto tray',
  'ice cream gelato mackinac fdge',
  'ice cream cookiecrm dutch fud',
  'ice cream gelato butr pecan',
  'ice cream sorbet lemoncello',
  'ice cream gelato choc nd vegan'],
 ['ice cream gelato mackinac fdge',
  'ice cream creme de menthe chip',
  'ice cream gelato van bean',
  'ice cream gelato rd velvet',
  'ice cream gelato peanut btr',
  'ice cream gelato swiss choc',
  'ice cream gelato straw frsh']]

In [ ]:
inv_en = []
for lst in inv_lst:
    lst_en = model.encode(lst)
    inv_en.append(lst_en)
inv_en

In [23]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
device

device(type='cuda')

In [ ]:
po_en = []
for lst in pov_lst:
    lst_en = model.encode(lst)
    po_en.append(lst_en)

In [ ]:
for i in range(0,len(po_en)):
    sim = model.similarity(inv_en[i],po_en[i])
    print(sim)

In [ ]:
# Function to map each invoice to its most similar PO product
def map_invoice_to_po(inv_lst, pov_lst, model):
    mappings = []

    for inv_items, po_items in zip(inv_lst, pov_lst):
        # Encode both invoice and PO items
        inv_encoded = model.encode(inv_items)
        po_encoded = model.encode(po_items)
        
        row_mapping = {}
        used_po_indices = set()

        # Loop through each invoice item
        for i, inv_item in enumerate(inv_items):
            # Calculate similarities with all PO items
            similarities = np.dot(po_encoded, inv_encoded[i]) / (
                np.linalg.norm(po_encoded, axis=1) * np.linalg.norm(inv_encoded[i])
            )
            
            # Sort the PO indices by similarity score in descending order
            sorted_po_indices = np.argsort(similarities)[::-1]
            
            # Find the best match that hasn't been used yet
            for po_idx in sorted_po_indices:
                if po_idx not in used_po_indices:
                    best_po_item = po_items[po_idx]
                    row_mapping[inv_item] = best_po_item
                    used_po_indices.add(po_idx)  # Mark the PO item as used
                    break
            
            # If no match found, assign 'NA' (this case should rarely happen)
            if inv_item not in row_mapping:
                row_mapping[inv_item] = "NA"

        mappings.append(row_mapping)

    return mappings

# Run the function on your lists
result = map_invoice_to_po(inv_lst, pov_lst, model)

In [64]:
len(result)

946

In [65]:
result[:2]

[{'25103 ice cream mango sorbetto tray': 'ice cream mango sorbetto tray',
  'ice cream gelato mackinac fdge 35lt': 'ice cream gelato mackinac fdge',
  'ice cream cookiecrm dutch fud 35lt': 'ice cream cookiecrm dutch fud',
  'ice cream gelato butr pecan 3 5lt': 'ice cream gelato butr pecan',
  'ice cream sorbet lemoncello 35 lt': 'ice cream sorbet lemoncello',
  'ice cream gelato choc nd vegan 35lt': 'ice cream gelato choc nd vegan'},
 {'ice cream gelato mackinac fdge 35lt': 'ice cream gelato mackinac fdge',
  'ice cream creme de minthe chip 35lt': 'ice cream creme de menthe chip',
  'ice cream gelato van bean 3 5lt': 'ice cream gelato van bean',
  'ice cream gelato rd velvet 35lt': 'ice cream gelato rd velvet',
  'ice cream gelato peanut btr 35lt': 'ice cream gelato peanut btr',
  'ice cream gelato swiss choc 3 5lt': 'ice cream gelato swiss choc',
  'ice cream gelato straw frsh 35lt': 'ice cream gelato straw frsh'}]

In [73]:
import pandas as pd

# Function to calculate exact match accuracy and return mismatches
def calculate_exact_accuracy_and_mismatches(result, true_mapping_clean):
    total_mappings = 0
    correct_mappings = 0
    mismatches = []

    # Iterate through each row to compare predicted vs true mapping
    for i, predicted_mapping in enumerate(result):
        true_mapping = true_mapping_clean[i]  # Get the ground truth mapping for this row
        
        # Loop through each predicted invoice-item pair
        for inv_item, po_item in predicted_mapping.items():
            total_mappings += 1
            # Check if the current inv_item -> po_item exists in true_mapping
            is_match = any(true_pair.get(inv_item) == po_item for true_pair in true_mapping)
            
            if is_match:
                correct_mappings += 1
            else:
                # Collect mismatched rows (invoice, predicted PO, true PO)
                for true_pair in true_mapping:
                    if inv_item in true_pair:
                        mismatches.append({
                            'invoice_item': inv_item,
                            'predicted_po_item': po_item,
                            'true_po_item': true_pair[inv_item],
                            'row_index': i
                        })
                        break

    # Calculate accuracy
    accuracy = correct_mappings / total_mappings if total_mappings > 0 else 0

    # Return accuracy and mismatches
    return accuracy, pd.DataFrame(mismatches)

# Assuming 'result' contains the predicted mappings and 'df["true_mapping_clean"]' has the true mappings
accuracy, mismatches_df = calculate_exact_accuracy_and_mismatches(result, df["true_mapping_clean"])

# Print the exact match accuracy
print(f"Exact Match Accuracy: {accuracy * 100:.2f}%")

Exact Match Accuracy: 71.61%


In [75]:
# Display the mismatches DataFrame
print("Mismatches:")
mismatches_df

Mismatches:


,invoice_item,predicted_po_item,true_po_item,row_index
0,scotchbrite quick clean griddle liquid 70040 3...,pad scrub stnls 175oz,cleaner grl quickclean pckt,2
1,scotchbrite stainless steel scrubbe,cleaner grl quickclean pckt,pad scrub stnls 175oz,2
2,hc plus apple juice cups 964oz,juice cranberry 15 ss cup,juice apple shelf stable,5
3,hc plus 15 cranberry juice cups 964oz,juice prune 100 shelf stable,juice cranberry 15 ss cup,5
4,hc plus prune juice cups 964oz,juice apple shelf stable,juice prune 100 shelf stable,5
...,...,...,...,...
1580,salmon 7 oz 10,squid ring tent,salmon fil prtn 78 oz raw ivp,942
1581,pork butt bone in boxed 1x4 inch,pork butt bnls 1x4 gap,pork butt bi bxd 1x4,943
1582,pork butt boneless 1x4,pork butt bi bxd 1x4,pork butt bnls 1x4 gap,943
1583,napkin band burgundy 15 x 425 sysco,band napkin papr 15x425 wht,band napkin papr 15x425 burg,945
